# Formación de sintagmas

*********************************************

Variables:

$w=[w_1, ..., w_n] \rightarrow$ El sintagma

* $e_t \in \{0,1\} \rightarrow$ Ocurrencia del evento en el momento $t$
* $s_t \in \{0,1\} \rightarrow$ Existencia de la situación en el momento $t$
* $s_{t-1} \rightarrow$ Situación en el momento $t-1$
* $w_t^i \rightarrow$ El evento del sintagma siendo formado en el momento $t$:
    * $i=0$: hipótesis de que el sintagma no se ha formado aún
    * $i=1$: hipótesis de que el sintagma se ha formado
* $\pi_t \rightarrow$ Observación

\begin{matrix}
    \pi^1 = \left\{
        \begin{matrix} 
            \frac{n(w_1...w_n)}{max(n(w_1), ..., n(w_n))} & if\ \max(n(w_1), ..., n(w_n)) > 0
                \\
            0 & otherwise 
        \end{matrix}\right.
    \\
    \pi^0=1-\pi^1
\end{matrix}

**La calidad *q* del sintagma no se va a tener en cuenta**


*********************************************

* $\alpha_j^i(t)=P[w_t=i|s_t=j]$

* $\beta_{kh}^j(t)=P[s_t=j|s_{t-1}=k,e_t=h]$

* $\nu^k(t)=P[s_{t-1}=k]$

* $\gamma^h(t)=P[e_t=h]$

* $\tau_i^{jkh}=P(s_t=j,s_{t-1}=k, e_t=h|w=i)$

$i, j, k, h \in \{0,1\}$

*********************************************

In [1]:
import json

EXT = ".json"
ALL_MONTHS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
LAST_DECADE = [2019, 2020, 2021, 2022, 2023]

DATA_PATH = "data/"

RESPONSE = "response"
DOCS = "docs"

ABSTRACT = "abstract"
SNIPPET = "snippet"
LEAD_PAR = "lead_paragraph"
PUB_DATE = "pub_date"

In [ ]:
years = list()
months = list()

years.extend(LAST_DECADE)
months.extend(ALL_MONTHS)

query = " lock down "

for year in years:    
    for month in months:
        
        path = DATA_PATH + str(year) + "/" + str(month) + EXT
        
        with open(path, "r") as f:
            docs = json.loads(f.read())[RESPONSE][DOCS]
            
            for doc in docs:
                
                if query in doc[ABSTRACT] or query in doc[SNIPPET] or query in doc[LEAD_PAR]:
                    print(f"[INFO] Articulo del año {doc[PUB_DATE][:10]}")
                    print(doc[ABSTRACT])
                    print(doc[SNIPPET])
                    print(doc[LEAD_PAR])
                    print()
                    print()
            


Consideraciones previas:

- La situación no se ha dado en $t=0$
    - $\nu^0(t)=1$
    - $\nu^1(t)=0$

- $\psi$: lista que, para cada t, contiene la probabilidad de que el evento que causa el sintagma se genere

- $\alpha, \beta, \nu, \gamma$ son listas de:

    - n listas, siendo n el subíndice:
        - cada lista de m posiciones según el superíndice.



- $\alpha$: [[ , ], [ , ]]
- $\beta$: [[ [ , ] , [ , ] ], [ [ , ] , [ , ] ]]
- $\nu$: una lista de dos posiciones
- $\gamma$: una lista de dos posiciones

- $\tau$: ocho listas de dos posiciones

In [ ]:
import random


def computeTau(alpha, beta, nu, gamma):
    
    tau = list([[], [], [], [], [], [], [], []])
    
    denominador = list([0, 0])  # lista de dos posiciones que se indexa con i
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for h in range(2):
                    denominador[i] += alpha[j][i] * beta[j][k*2 + h*1] * nu[k] * (t - 1) * gamma[h]

    for i in range(2):
        for j in range(2):
            for k in range(2):
                for h in range(2):
                    
                    numerador = alpha[j][i] * beta[j][k*2 + h*1] * nu[k] * (t - 1) * gamma[h]

                    total = numerador / denominador[i]
                    tau[j*4 + k*2 + h*1].append(total)
    return tau


psi = list()
nu = list([1, 0])
time_max = 6


for t in range(2, time_max):

    alpha = list([[], []])
    beta = list([[], []])
    gamma = list()
    tau = list([[], [], [], [], [], [], [], []])

    # TODO: calcular pi
    pi = list([0.8, 0.2])

    for i in range(2):
        alpha[0].append(random.random())
        alpha[1].append(random.random())

        beta[0].append(random.random())
        beta[0].append(random.random())
        beta[1].append(random.random())
        beta[1].append(random.random())

        gamma.append(random.random())

    # TODO: convergencia

    tau = computeTau(alpha, beta, nu, gamma)


    # ---------------------------------------------------
    # TODO: calcular alpha

    for j in range(2):
        for k in range(2):
                for h in range(2):
                    pass
 
    # ---------------------------------------------------


    # ---------------------------------------------------
    # TODO: calcular beta
 
    # ---------------------------------------------------


    # ---------------------------------------------------
    # TODO: calcular gamma
 
    # ---------------------------------------------------


print("[INFO] Alpha", alpha)
print("[INFO] Beta", beta)
print("[INFO] Gamma", gamma)
print()

for n, i in enumerate(tau):
    print(f"[INFO] Tau {n}", i)
        
    
    

In [2]:
import random


psi = list()
nu = list([1, 0])
time_max = 6


for t in range(2, time_max):
    
    alpha = list([[], []])
    beta = list([[], []])
    gamma = list()
    # tau = list()

    # TODO: calcular pi
    pi = list([0.8, 0.2])

    for i in range(2):
        alpha[0].append(random.random())
        alpha[1].append(random.random())

        beta[0].append(random.random())
        beta[0].append(random.random())
        beta[1].append(random.random())
        beta[1].append(random.random())

        gamma.append(random.random())



print("[INFO] Alpha", alpha)
print("[INFO] Beta", beta)
print("[INFO] Gamma", gamma)
print()

# for n, i in enumerate(tau):
#     print(f"[INFO] Tau {n}", i)

[INFO] Alpha [[0.9293523433740217, 0.15418298655973606], [0.19518571486366232, 0.305467315687421]]
[INFO] Beta [[0.03708969226817216, 0.22995631946844108, 0.7241779087631127, 0.5340038340583395], [0.8075812544948782, 0.7316204536138221, 0.8026945586450113, 0.44773356456952995]]
[INFO] Gamma [0.17160652975505963, 0.47636358562377956]



In [124]:
import numpy as np

beta = np.random.rand(2, 2, 2)
# print("\nbeta:")
# print("\n", beta)

gamma = np.random.rand(2)
print("\ngamma:")
print("\n", gamma)

pi = np.random.dirichlet(np.ones(2))
print("\n--> pi:")
print("\n", pi)

tau = np.random.rand(2, 2, 2, 2)
# print("\ntau:")
# print("\n", tau)

alpha = np.random.rand(2, 2)
# print("\n--> alpha:")
# print("\n", alpha)

nu = np.array([1, 0])
print(nu)

print("\n------------------------------------------\n")


def compute_alpha(alpha, tau):

    for i, level_zero in enumerate(alpha):
        for j, _ in enumerate(level_zero):

            numerador = pi[i] * tau[i][j].sum()
            
            alpha[i][j] = numerador

    return alpha


def compute_beta(beta, tau):

    for i, level_zero in enumerate(beta):
        for j, level_one in enumerate(level_zero):
            for k, _ in enumerate(level_one):
                
                numerador = 0
                for h, _ in enumerate(pi):
                    numerador += pi[h] * tau[i][j][k][h]
                
                beta[i][j][k] = numerador

    return beta


def compute_gamma(gamma, tau):
    
    for i, _ in enumerate(gamma):

        numerador = 0
        for j, level_one in enumerate(tau):
            for k, level_two in enumerate(level_one):
                for h, _ in enumerate(level_two):
                    numerador += pi[j] * tau[i][j][k][h]

        gamma[i] = numerador
    
    return gamma
    

def compute_tau(tau, alpha, beta, nu, gamma, t):
    
    for i, level_zero in enumerate(tau):
        for j, level_one in enumerate(level_zero):
            for k, level_two in enumerate(level_one):
                for h, _ in enumerate(level_two):

                    numerador = alpha[i][j] * beta[j][k][h] * nu[k] * (t - 1) * gamma[h]
                    tau[i][j][k][h] = numerador
    return tau


times = 10
for t in range(2, times):

    tau = compute_tau(tau, alpha, beta, nu, gamma, t)
    alpha = compute_alpha(alpha, tau)
    beta = compute_beta(beta, tau)
    gamma = compute_gamma(gamma, tau)




gamma:

 [0.91469927 0.87601722]

--> pi:

 [0.46009079 0.53990921]
[1 0]

------------------------------------------

